In [4]:
import tkinter as tk
from tkinter import ttk
import tkcalendar
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import joblib
from prophet import Prophet
import xgboost as xgb
import os

# Load saved models and scaler
model_dir = "model_files_02_oct"  # Replace with your model directory path
prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
xgb_model = xgb.XGBRegressor().load_model(os.path.join(model_dir, "xgb_model.json"))
scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))


def get_last_regressor_values():
    # Assuming your data has the last date information
    last_date = pd.read_csv("df_interpolated (4).csv")["date"].iloc[-1]
    last_date = datetime.strptime(last_date, "%d/%m/%Y")
    df = pd.DataFrame({col: 0 for col in regressors})
    df["ds"] = last_date
    return df.to_dict(orient="records")[0]


def make_future_residual_predictions(last_sequence, num_predictions=1):
    future_residuals = []
    current_seq = last_sequence

    for _ in range(num_predictions):
        future_pred_residual = xgb_model.predict(np.array([current_seq]))
        future_residuals.append(future_pred_residual[0])

        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual

    return future_residuals


def forecast_gold_price(selected_date):
    # Convert selected date to datetime
    selected_date = datetime.strptime(selected_date, "%Y-%m-%d")

    # Get last available regressor values
    last_regressor_values = get_last_regressor_values()

    # Create a DataFrame for the selected date with regressor values
    future_data = pd.DataFrame({col: last_regressor_values[col] for col in regressors})
    future_data["ds"] = selected_date

    # Forecast using Prophet
    prophet_forecast = prophet_model.predict(future_data)
    prophet_prediction = prophet_forecast["yhat"][0]

    # Predict residuals using XGBoost
    last_sequence = scaler.transform(np.array([future_data.drop("ds", axis=1)]).reshape(1, -1))
    future_residual_pred = make_future_residual_predictions(last_sequence)[0]

    # Combine predictions
    final_prediction = prophet_prediction + abs(future_residual_pred)

    # Display prediction
    prediction_label.config(text=f"Predicted Gold Price (LKR): {final_prediction:.2f}")


def on_select_date(selected_date):
    # Clear previous prediction
    prediction_label.config(text="")
    forecast_gold_price(selected_date.strftime("%Y-%m-%d"))


root = tk.Tk()
root.title("Gold Price Forecast")

# Calendar frame
calendar_frame = ttk.Frame(root)
calendar_frame.pack(padx=10, pady=10)

# Calendar widget
calendar = tkcalendar.Calendar(calendar_frame, selectmode="day", date_pattern="y-%m-%d")
calendar.pack(pady=10)
calendar.bind("<<DateSelect>>", on_select_date)

# Prediction label
prediction_label = ttk.Label(root, text="")
prediction_label.pack(pady=10)

# Run the main loop
root.mainloop()

In [10]:
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from tkinter import messagebox
import pandas as pd
import joblib
import os
from datetime import datetime

# Load the trained Prophet model
model_dir = "model_files_02_oct"
prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))

# Define the regressors your model expects (use dummy values for now)
regressors = ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']

# Create the GUI window
class ForecastApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Gold Price Forecast")
        self.root.geometry("400x400")

        # Create a label for instructions
        self.label = ttk.Label(self.root, text="Select a date to forecast the gold price:")
        self.label.pack(pady=10)

        # Create the Calendar widget
        self.cal = Calendar(self.root, selectmode='day', year=2023, month=10, day=1)
        self.cal.pack(pady=20)

        # Create a button to confirm the selection and forecast
        self.button = ttk.Button(self.root, text="Forecast Gold Price", command=self.forecast_price)
        self.button.pack(pady=20)

        # Create a label to display the forecast result
        self.result_label = ttk.Label(self.root, text="")
        self.result_label.pack(pady=20)

    def forecast_price(self):
        # Get the selected date from the calendar
        selected_date = self.cal.get_date()

        # Convert the selected date to a Pandas datetime object
        forecast_date = pd.to_datetime(selected_date)

        # Check if the date is in the past
        today = pd.Timestamp(datetime.today().date())
        if forecast_date <= today:
            messagebox.showerror("Invalid Date", "Please select a future date.")
            return

        # Prepare the future DataFrame for Prophet model prediction
        future = pd.DataFrame({'ds': [forecast_date]})

        # Add dummy regressor values (you can set them to any constant value)
        for regressor in regressors:
            future[regressor] = 0  # or any reasonable dummy value, like the mean of historical data

        # Get Prophet model prediction
        prophet_forecast = prophet_model.predict(future)
        prophet_predicted_value = prophet_forecast['yhat'].values[0]

        # Display the forecast result in the GUI
        self.result_label.config(text=f"Forecasted Gold Price (LKR): {prophet_predicted_value:.2f}")


# Run the Tkinter app
if __name__ == "__main__":
    root = tk.Tk()
    app = ForecastApp(root)
    root.mainloop()


In [38]:
import pandas as pd
import numpy as np
import xgboost as xgb
from prophet import Prophet
import joblib
import logging
import os
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the saved models and other necessary components
model_dir = "model_files_02_oct"

# Load the saved Prophet model
prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
logging.info("Prophet model loaded successfully.")

# Load the saved XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.load_model(os.path.join(model_dir, "xgb_model.json"))
logging.info("XGBoost model loaded successfully.")

# Load the saved StandardScaler
scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
logging.info("Scaler loaded successfully.")

# Load the regressors
regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))
logging.info("Regressors loaded successfully: %s", regressors)

# Load the dataset
df = pd.read_csv('df_interpolated (4).csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Rename columns for Prophet
df = df.rename(columns={'date': 'ds', 'gold_lkr': 'y'})

# Create lag features for the residuals
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Apply the lag feature creation on the residuals column
def prepare_data_with_lags(df):
    # Forecast the trend with Prophet
    df_forecast = prophet_model.predict(df[['ds'] + regressors])
    
    # Calculate residuals (actual - forecast)
    df['residuals'] = df['y'] - df_forecast['yhat']
    
    # Create lag features for the residuals
    df_with_lags = create_lag_features(df, lags=90, target_col='residuals')
    
    # Drop rows with NaN values caused by shifting (lags)
    df_with_lags.dropna(inplace=True)
    
    return df_with_lags

# Function to make future residual predictions using XGBoost
def make_future_residual_predictions(last_sequence, model, num_predictions=90):
    future_residuals = []
    current_seq = last_sequence
    
    for _ in range(num_predictions):
        # Ensure that the correct number of features (regressors + lags) are passed to the model
        if len(current_seq) != scaler.n_features_in_:
            logging.warning(f"Feature mismatch: Expected {scaler.n_features_in_}, but got {len(current_seq)}.")
            # If necessary, pad the sequence with zeros (or handle appropriately)
            current_seq = np.pad(current_seq, (0, scaler.n_features_in_ - len(current_seq)), 'constant')
        
        current_seq_scaled = scaler.transform([current_seq])
        future_pred_residual = model.predict(current_seq_scaled)
        future_residuals.append(future_pred_residual[0])
        
        # Shift the lag sequence and append the new residual
        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual #########################################

    return future_residuals

# Function to handle forecast
def forecast_up_to_date(selected_date):
    # Prepare the data with lags
    df_with_lags = prepare_data_with_lags(df)
    
    # Convert the selected date to datetime
    selected_date = datetime.strptime(selected_date, "%Y-%m-%d")
    
    # Prophet forecast for future dates
    future_dates = pd.date_range(start=df['ds'].max(), end=selected_date, freq='B')
    future = pd.DataFrame(future_dates, columns=['ds'])
    
    # Use the last available regressor values for the future predictions
    last_regressor_values = df[regressors].iloc[-1].to_dict()
    for regressor in regressors:
        future[regressor] = last_regressor_values[regressor]

    # Prophet forecast for the selected future dates
    prophet_future_forecast = prophet_model.predict(future)
    
    # Create the last sequence of lag features (residuals) from the training data
    last_sequence = df_with_lags.iloc[-1][regressors + [f'lag_{i}' for i in range(1, 91)]].values   ########################
    
    # Predict residuals using XGBoost
    future_residuals_pred = make_future_residual_predictions(last_sequence, xgb_model, num_predictions=len(future_dates))

    # Combine Prophet's predictions with XGBoost's residuals
    future_predictions = prophet_future_forecast['yhat'] + future_residuals_pred

    # Return the forecasted values and the corresponding dates
    return pd.DataFrame({
        'Date': future_dates,
        'Prophet_Prediction': prophet_future_forecast['yhat'],
        'XGBoost_Residuals': future_residuals_pred,
        'Hybrid_Prediction': future_predictions
    })

# Function to handle the "Submit" button
def submit_forecast():
    selected_date = cal.get_date()
    forecast_df = forecast_up_to_date(selected_date)
    
    # Clear the text box before displaying new results
    result_text.delete(1.0, tk.END)
    
    # Display the forecasted results
    result_text.insert(tk.END, forecast_df.to_string(index=False))

# GUI setup
root = tk.Tk()
root.title("Gold Price Forecast System")

# Label for selecting a date
date_label = tk.Label(root, text="Select Date for Forecast:")
date_label.pack(pady=10)

# Calendar widget for date selection
cal = Calendar(root, selectmode='day', date_pattern='yyyy-mm-dd')
cal.pack(pady=10)

# Submit button
submit_btn = ttk.Button(root, text="Submit", command=submit_forecast)
submit_btn.pack(pady=10)

# Text box to display the forecast results
result_text = tk.Text(root, height=20, width=80)
result_text.pack(pady=10)

# Start the Tkinter event loop
root.mainloop()


2024-10-03 12:00:38,408 - INFO - Prophet model loaded successfully.
2024-10-03 12:00:38,510 - INFO - XGBoost model loaded successfully.
2024-10-03 12:00:38,517 - INFO - Scaler loaded successfully.
2024-10-03 12:00:38,518 - INFO - Regressors loaded successfully: ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']
2024-10-03 12:00:41,515 - WARNING - Feature mismatch: Expected 98, but got 97.
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardSca

In [16]:
from datetime import datetime, timedelta

# Convert the date to a datetime object
date = datetime(year=2024, month=10, day=14)

# Subtract 90 days from the date
date_90_days_back = date - timedelta(days=90)

# Print the result
print(date_90_days_back.strftime("%d/%m/%Y"))

16/07/2024


In [39]:
import pandas as pd
import numpy as np
import xgboost as xgb
from prophet import Prophet
import joblib
import logging
import os
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the saved models and other necessary components
model_dir = "model_files_02_oct"

# Load the saved Prophet model
prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
logging.info("Prophet model loaded successfully.")

# Load the saved XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.load_model(os.path.join(model_dir, "xgb_model.json"))
logging.info("XGBoost model loaded successfully.")

# Load the saved StandardScaler
scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
logging.info("Scaler loaded successfully.")

# Load the regressors
regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))
logging.info("Regressors loaded successfully: %s", regressors)

# Load the dataset
df = pd.read_csv('df_interpolated (4).csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Rename columns for Prophet
df = df.rename(columns={'date': 'ds', 'gold_lkr': 'y'})

# Create lag features for the residuals
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Apply the lag feature creation on the residuals column
def prepare_data_with_lags(df):
    # Forecast the trend with Prophet
    df_forecast = prophet_model.predict(df[['ds'] + regressors])
    
    # Calculate residuals (actual - forecast)
    df['residuals'] = df['y'] - df_forecast['yhat']
    
    # Create lag features for the residuals
    df_with_lags = create_lag_features(df, lags=90, target_col='residuals')
    
    # Drop rows with NaN values caused by shifting (lags)
    df_with_lags.dropna(inplace=True)
    
    return df_with_lags

# Function to make future residual predictions using XGBoost
def make_future_residual_predictions(last_sequence, model, num_predictions=90):
    future_residuals = []
    current_seq = last_sequence
    
    for _ in range(num_predictions):
        # Ensure that the correct number of features (regressors + lags) are passed to the model
        if len(current_seq) != scaler.n_features_in_:
            logging.warning(f"Feature mismatch: Expected {scaler.n_features_in_}, but got {len(current_seq)}.")
            # If necessary, pad the sequence with zeros (or handle appropriately)
            current_seq = np.pad(current_seq, (0, scaler.n_features_in_ - len(current_seq)), 'constant')
        
        current_seq_scaled = scaler.transform([current_seq])
        future_pred_residual = model.predict(current_seq_scaled)
        
        # Use the absolute value of residual predictions to avoid negative predictions
        future_residuals.append(abs(future_pred_residual[0]))
        
        # Shift the lag sequence and append the new residual
        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual

    return future_residuals

# Function to handle forecast
def forecast_up_to_date(selected_date):
    # Prepare the data with lags
    df_with_lags = prepare_data_with_lags(df)
    
    # Convert the selected date to datetime
    selected_date = datetime.strptime(selected_date, "%Y-%m-%d")
    
    # Prophet forecast for future dates
    future_dates = pd.date_range(start=df['ds'].max(), end=selected_date, freq='B')
    future = pd.DataFrame(future_dates, columns=['ds'])
    
    # Use the last available regressor values for the future predictions
    last_regressor_values = df[regressors].iloc[-1].to_dict()
    for regressor in regressors:
        future[regressor] = last_regressor_values[regressor]

    # Prophet forecast for the selected future dates
    prophet_future_forecast = prophet_model.predict(future)
    
    # Create the last sequence of lag features (residuals) from the training data
    last_sequence = df_with_lags.iloc[-1][regressors + [f'lag_{i}' for i in range(1, 91)]].values
    
    # Predict residuals using XGBoost
    future_residuals_pred = make_future_residual_predictions(last_sequence, xgb_model, num_predictions=len(future_dates))

    # Combine Prophet's predictions with XGBoost's residuals
    future_predictions = prophet_future_forecast['yhat'] + future_residuals_pred
    
    # Apply the final formula transformation ((future_predictions / 31.1035) * 8)
    final_predictions = (future_predictions / 31.1035) * 8

    # Return the forecasted values and the corresponding dates
    return pd.DataFrame({
        'Date': future_dates,
        'Prophet_Prediction': prophet_future_forecast['yhat'],
        'XGBoost_Residuals': future_residuals_pred,
        'Hybrid_Prediction': future_predictions,
        'Final_Prediction': final_predictions
    })

# Function to handle the "Submit" button
def submit_forecast():
    selected_date = cal.get_date()
    forecast_df = forecast_up_to_date(selected_date)
    
    # Clear the text box before displaying new results
    result_text.delete(1.0, tk.END)
    
    # Display the forecasted results
    result_text.insert(tk.END, forecast_df.to_string(index=False))

# GUI setup
root = tk.Tk()
root.title("Gold Price Forecast System")

# Label for selecting a date
date_label = tk.Label(root, text="Select Date for Forecast:")
date_label.pack(pady=10)

# Calendar widget for date selection
cal = Calendar(root, selectmode='day', date_pattern='yyyy-mm-dd')
cal.pack(pady=10)

# Submit button
submit_btn = ttk.Button(root, text="Submit", command=submit_forecast)
submit_btn.pack(pady=10)

# Text box to display the forecast results
result_text = tk.Text(root, height=20, width=80)
result_text.pack(pady=10)

# Start the Tkinter event loop
root.mainloop()


2024-10-03 12:10:17,944 - INFO - Prophet model loaded successfully.
2024-10-03 12:10:18,047 - INFO - XGBoost model loaded successfully.
2024-10-03 12:10:18,054 - INFO - Scaler loaded successfully.
2024-10-03 12:10:18,054 - INFO - Regressors loaded successfully: ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']
2024-10-03 12:10:21,274 - WARNING - Feature mismatch: Expected 98, but got 97.
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardSca

In [48]:
import pandas as pd
import numpy as np
import xgboost as xgb
from prophet import Prophet
import joblib
import logging
import os
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load the saved models and other necessary components
model_dir = "model_files_02_oct"

# Load the saved Prophet model
prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
logging.info("Prophet model loaded successfully.")

# Load the saved XGBoost model with a fixed random state
xgb_model = xgb.XGBRegressor(random_state=42)  # Ensure consistency by fixing random state
xgb_model.load_model(os.path.join(model_dir, "xgb_model.json"))
logging.info("XGBoost model loaded successfully.")

# Load the saved StandardScaler
scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
logging.info("Scaler loaded successfully.")

# Load the regressors
regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))
logging.info("Regressors loaded successfully: %s", regressors)

# Load the dataset
df = pd.read_csv('df_interpolated (4).csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Rename columns for Prophet
df = df.rename(columns={'date': 'ds', 'gold_lkr': 'y'})

# Create lag features for the residuals
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Apply the lag feature creation on the residuals column
def prepare_data_with_lags(df):
    # Forecast the trend with Prophet
    df_forecast = prophet_model.predict(df[['ds'] + regressors])
    
    # Calculate residuals (actual - forecast)
    df['residuals'] = df['y'] - df_forecast['yhat']
    
    # Create lag features for the residuals
    df_with_lags = create_lag_features(df, lags=90, target_col='residuals')
    
    # Drop rows with NaN values caused by shifting (lags)
    df_with_lags.dropna(inplace=True)
    
    return df_with_lags

# Function to make future residual predictions using XGBoost
def make_future_residual_predictions(last_sequence, model, num_predictions=90):
    future_residuals = []
    current_seq = last_sequence
    
    for _ in range(num_predictions):
        # Ensure that the correct number of features (regressors + lags) are passed to the model
        if len(current_seq) != scaler.n_features_in_:
            logging.warning(f"Feature mismatch: Expected {scaler.n_features_in_}, but got {len(current_seq)}.")
            # If necessary, pad the sequence with zeros (or handle appropriately)
            current_seq = np.pad(current_seq, (0, scaler.n_features_in_ - len(current_seq)), 'constant')
        
        current_seq_scaled = scaler.transform([current_seq])
        future_pred_residual = model.predict(current_seq_scaled)
        
        # Instead of absolute values, use exact residual values for consistency
        future_residuals.append(future_pred_residual[0])
        
        # Shift the lag sequence and append the new residual
        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual

    return future_residuals

# Function to handle forecast
def forecast_up_to_date(selected_date):
    # Prepare the data with lags
    df_with_lags = prepare_data_with_lags(df)
    
    # Convert the selected date to datetime
    selected_date = datetime.strptime(selected_date, "%Y-%m-%d")
    
    # Prophet forecast for future dates
    future_dates = pd.date_range(start=df['ds'].max(), end=selected_date, freq='B')
    future = pd.DataFrame(future_dates, columns=['ds'])
    
    # Use the last available regressor values for the future predictions
    last_regressor_values = df[regressors].iloc[-1].to_dict()
    for regressor in regressors:
        future[regressor] = last_regressor_values[regressor]

    # Prophet forecast for the selected future dates
    prophet_future_forecast = prophet_model.predict(future)
    
    # Create the last sequence of lag features (residuals) from the training data
    last_sequence = df_with_lags.iloc[-1][regressors + [f'lag_{i}' for i in range(1, 91)]].values
    
    # Predict residuals using XGBoost
    future_residuals_pred = make_future_residual_predictions(last_sequence, xgb_model, num_predictions=len(future_dates))

    # Combine Prophet's predictions with XGBoost's residuals
    future_predictions = prophet_future_forecast['yhat'] + future_residuals_pred
    
    # Apply the final formula transformation ((future_predictions / 31.1035) * 8)
    final_predictions = (future_predictions / 31.1035) * 8

    # Return the forecasted values and the corresponding dates
    return pd.DataFrame({
        'Date': future_dates,
        'Prophet_Prediction': prophet_future_forecast['yhat'],
        'XGBoost_Residuals': future_residuals_pred,
        'Hybrid_Prediction': future_predictions,
        'Final_Prediction': final_predictions
    })

# Function to handle the "Submit" button
def submit_forecast():
    selected_date = cal.get_date()
    forecast_df = forecast_up_to_date(selected_date)
    
    # Clear the text box before displaying new results
    result_text.delete(1.0, tk.END)
    
    # Display the forecasted results
    result_text.insert(tk.END, forecast_df.to_string(index=False))

# GUI setup
root = tk.Tk()
root.title("Gold Price Forecast System")

# Label for selecting a date
date_label = tk.Label(root, text="Select Date for Forecast:")
date_label.pack(pady=10)

# Calendar widget for date selection
cal = Calendar(root, selectmode='day', date_pattern='yyyy-mm-dd')
cal.pack(pady=10)

# Submit button
submit_btn = ttk.Button(root, text="Submit", command=submit_forecast)
submit_btn.pack(pady=10)

# Text box to display the forecast results
result_text = tk.Text(root, height=70, width=100)
result_text.pack(pady=10)

# Start the Tkinter event loop
root.mainloop()


2024-10-03 12:25:37,414 - INFO - Prophet model loaded successfully.
2024-10-03 12:25:37,514 - INFO - XGBoost model loaded successfully.
2024-10-03 12:25:37,519 - INFO - Scaler loaded successfully.
2024-10-03 12:25:37,520 - INFO - Regressors loaded successfully: ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']
2024-10-03 12:25:42,626 - WARNING - Feature mismatch: Expected 98, but got 97.
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardSca

## GUI FINAL

In [56]:
import pandas as pd
import numpy as np
import xgboost as xgb
from prophet import Prophet
import joblib
import logging
import os
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define initial values for the global variables
model_dir = "model_files_02_oct"
prophet_model = None
xgb_model = None
scaler = None
regressors = None
df = None

# Cache to store the results for already forecasted dates
forecast_cache = {}

# Function to reset all global variables to their initial states
def reset_variables():
    global prophet_model, xgb_model, scaler, regressors, df, forecast_cache
    # Reload the saved models and other necessary components
    prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
    logging.info("Prophet model loaded successfully.")

    # Load the saved XGBoost model with a fixed random state
    xgb_model = xgb.XGBRegressor(random_state=42)  # Ensure consistency by fixing random state
    xgb_model.load_model(os.path.join(model_dir, "xgb_model.json"))
    logging.info("XGBoost model loaded successfully.")

    # Load the saved StandardScaler
    scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
    logging.info("Scaler loaded successfully.")

    # Load the regressors
    regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))
    logging.info("Regressors loaded successfully: %s", regressors)

    # Load the dataset
    df = pd.read_csv('df_interpolated (4).csv')
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

    # Rename columns for Prophet
    df.rename(columns={'date': 'ds', 'gold_lkr': 'y'}, inplace=True)

    # Clear forecast cache for new forecasts
    forecast_cache = {}

# Function to create lag features for residuals
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Function to apply lag feature creation on the residuals column
def prepare_data_with_lags(df):
    # Forecast the trend with Prophet
    df_forecast = prophet_model.predict(df[['ds'] + regressors])
    
    # Calculate residuals (actual - forecast)
    df['residuals'] = df['y'] - df_forecast['yhat']
    
    # Create lag features for the residuals
    df_with_lags = create_lag_features(df, lags=90, target_col='residuals')
    
    # Drop rows with NaN values caused by shifting (lags)
    df_with_lags.dropna(inplace=True)
    
    return df_with_lags

# Function to make future residual predictions using XGBoost
def make_future_residual_predictions(last_sequence, model, num_predictions=90):
    future_residuals = []
    current_seq = last_sequence
    
    for _ in range(num_predictions):
        # Ensure that the correct number of features (regressors + lags) are passed to the model
        if len(current_seq) != scaler.n_features_in_:
            logging.warning(f"Feature mismatch: Expected {scaler.n_features_in_}, but got {len(current_seq)}.")
            # If necessary, pad the sequence with zeros (or handle appropriately)
            current_seq = np.pad(current_seq, (0, scaler.n_features_in_ - len(current_seq)), 'constant')
        
        current_seq_scaled = scaler.transform([current_seq])
        future_pred_residual = model.predict(current_seq_scaled)
        
        # Instead of absolute values, use exact residual values for consistency
        future_residuals.append(future_pred_residual[0])
        
        # Shift the lag sequence and append the new residual
        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual

    return future_residuals

# Function to handle forecast
def forecast_up_to_date(selected_date):
    # Prepare the data with lags
    df_with_lags = prepare_data_with_lags(df)
    
    # Convert the selected date to datetime
    selected_date = datetime.strptime(selected_date, "%Y-%m-%d")
    
    # Prophet forecast for future dates
    future_dates = pd.date_range(start=df['ds'].max(), end=selected_date, freq='B')
    future = pd.DataFrame(future_dates, columns=['ds'])
    
    # Use the last available regressor values for the future predictions
    last_regressor_values = df[regressors].iloc[-1].to_dict()
    for regressor in regressors:
        future[regressor] = last_regressor_values[regressor]

    # Prophet forecast for the selected future dates
    prophet_future_forecast = prophet_model.predict(future)
    
    # Create the last sequence of lag features (residuals) from the training data
    last_sequence = df_with_lags.iloc[-1][regressors + [f'lag_{i}' for i in range(1, 91)]].values
    
    # Predict residuals using XGBoost
    future_residuals_pred = make_future_residual_predictions(last_sequence, xgb_model, num_predictions=len(future_dates))

    # Combine Prophet's predictions with XGBoost's residuals
    future_predictions = prophet_future_forecast['yhat'] + abs(future_residuals_pred)
    
    # Apply the final formula transformation ((future_predictions / 31.1035) * 8)
    final_predictions = (future_predictions / 31.1035) * 8

    # Return the forecasted values and the corresponding dates
    return pd.DataFrame({
        'Date': future_dates,
        'Prophet_Prediction': prophet_future_forecast['yhat'],
        'XGBoost_Residuals': future_residuals_pred,
        'Hybrid_Prediction': future_predictions,
        'Final_Prediction': final_predictions
    })

# Function to handle the "Submit" button
def submit_forecast():
    selected_date = cal.get_date()
    forecast_df = forecast_up_to_date(selected_date)
    
    # Clear the text box before displaying new results
    result_text.delete(1.0, tk.END)
    
    # Display the forecasted results
    result_text.insert(tk.END, forecast_df.to_string(index=False))
    
    # Reset all variables to their initial states after the forecast
    reset_variables()

# GUI setup
root = tk.Tk()
root.title("Gold Price Forecast System")

# Label for selecting a date
date_label = tk.Label(root, text="Select Date for Forecast:")
date_label.pack(pady=10)

# Calendar widget for date selection
cal = Calendar(root, selectmode='day', date_pattern='yyyy-mm-dd')
cal.pack(pady=10)

# Submit button
submit_btn = ttk.Button(root, text="Submit", command=submit_forecast)
submit_btn.pack(pady=10)

# Text box to display the forecast results
result_text = tk.Text(root, height=70, width=100)
result_text.pack(pady=10)

# Initialize the models and variables for the first time
reset_variables()

# Start the Tkinter event loop
root.mainloop()


2024-10-03 15:05:58,188 - INFO - Prophet model loaded successfully.
2024-10-03 15:05:58,416 - INFO - XGBoost model loaded successfully.
2024-10-03 15:05:58,432 - INFO - Scaler loaded successfully.
2024-10-03 15:05:58,432 - INFO - Regressors loaded successfully: ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']
2024-10-03 15:06:01,774 - WARNING - Feature mismatch: Expected 98, but got 97.
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardSca

## FINAL GUI CODE 2

In [59]:
import pandas as pd
import numpy as np
import xgboost as xgb
from prophet import Prophet
import joblib
import logging
import os
import tkinter as tk
from tkinter import ttk
from tkcalendar import Calendar
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define initial values for the global variables
model_dir = "model_files_02_oct"
prophet_model = None
xgb_model = None
scaler = None
regressors = None
df = None

# Cache to store the results for already forecasted dates
forecast_cache = {}

# Function to reset all global variables to their initial states
def reset_variables():
    global prophet_model, xgb_model, scaler, regressors, df, forecast_cache
    # Reload the saved models and other necessary components
    prophet_model = joblib.load(os.path.join(model_dir, "prophet_model.pkl"))
    logging.info("Prophet model loaded successfully.")

    # Load the saved XGBoost model with a fixed random state
    xgb_model = xgb.XGBRegressor(random_state=42)  # Ensure consistency by fixing random state
    xgb_model.load_model(os.path.join(model_dir, "xgb_model.json"))
    logging.info("XGBoost model loaded successfully.")

    # Load the saved StandardScaler
    scaler = joblib.load(os.path.join(model_dir, "scaler.pkl"))
    logging.info("Scaler loaded successfully.")

    # Load the regressors
    regressors = joblib.load(os.path.join(model_dir, "regressors.pkl"))
    logging.info("Regressors loaded successfully: %s", regressors)

    # Load the dataset
    df = pd.read_csv('df_interpolated (4).csv')
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

    # Rename columns for Prophet
    df.rename(columns={'date': 'ds', 'gold_lkr': 'y'}, inplace=True)

    # Clear forecast cache for new forecasts
    forecast_cache = {}

# Function to create lag features for residuals
def create_lag_features(data, lags, target_col):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data[target_col].shift(lag)
    return data

# Function to apply lag feature creation on the residuals column
def prepare_data_with_lags(df):
    # Forecast the trend with Prophet
    df_forecast = prophet_model.predict(df[['ds'] + regressors])
    
    # Calculate residuals (actual - forecast)
    df['residuals'] = df['y'] - df_forecast['yhat']
    
    # Create lag features for the residuals
    df_with_lags = create_lag_features(df, lags=90, target_col='residuals')
    
    # Drop rows with NaN values caused by shifting (lags)
    df_with_lags.dropna(inplace=True)
    
    return df_with_lags

# Function to make future residual predictions using XGBoost
def make_future_residual_predictions(last_sequence, model, num_predictions=90):
    future_residuals = []
    current_seq = last_sequence
    
    for _ in range(num_predictions):
        # Ensure that the correct number of features (regressors + lags) are passed to the model
        if len(current_seq) != scaler.n_features_in_:
            logging.warning(f"Feature mismatch: Expected {scaler.n_features_in_}, but got {len(current_seq)}.")
            # If necessary, pad the sequence with zeros (or handle appropriately)
            current_seq = np.pad(current_seq, (0, scaler.n_features_in_ - len(current_seq)), 'constant')
        
        current_seq_scaled = scaler.transform([current_seq])
        future_pred_residual = model.predict(current_seq_scaled)
        
        # Append the predicted residuals
        future_residuals.append(future_pred_residual[0])
        
        # Shift the lag sequence and append the new residual
        current_seq = np.roll(current_seq, -1)
        current_seq[-1] = future_pred_residual

    return future_residuals

# Function to handle forecast
def forecast_up_to_date(selected_date):
    # Prepare the data with lags
    df_with_lags = prepare_data_with_lags(df)
    
    # Convert the selected date to datetime
    selected_date = datetime.strptime(selected_date, "%Y-%m-%d")
    
    # Prophet forecast for future dates
    future_dates = pd.date_range(start=df['ds'].max(), end=selected_date, freq='B')
    future = pd.DataFrame(future_dates, columns=['ds'])
    
    # Use the last available regressor values for the future predictions
    last_regressor_values = df[regressors].iloc[-1].to_dict()
    for regressor in regressors:
        future[regressor] = last_regressor_values[regressor]

    # Prophet forecast for the selected future dates
    prophet_future_forecast = prophet_model.predict(future)
    
    # Create the last sequence of lag features (residuals) from the training data
    last_sequence = df_with_lags.iloc[-1][regressors + [f'lag_{i}' for i in range(1, 91)]].values
    
    # Predict residuals using XGBoost
    future_residuals_pred = make_future_residual_predictions(last_sequence, xgb_model, num_predictions=len(future_dates))

    # Combine Prophet's predictions with XGBoost's residuals without using abs()
    future_predictions = prophet_future_forecast['yhat'] + np.abs(future_residuals_pred)
    
    # Apply the final formula transformation ((future_predictions / 31.1035) * 8)
    final_predictions = (future_predictions / 31.1035) * 8

    # Return the forecasted values and the corresponding dates
    return pd.DataFrame({
        'Date': future_dates,
        'Prophet_Prediction': prophet_future_forecast['yhat'],
        'XGBoost_Residuals': future_residuals_pred,
        'Hybrid_Prediction': future_predictions,
        'Final_Prediction': final_predictions
    })

# Function to handle the "Submit" button
def submit_forecast():
    selected_date = cal.get_date()
    forecast_df = forecast_up_to_date(selected_date)
    
    # Clear the text box before displaying new results
    result_text.delete(1.0, tk.END)
    
    # Display the forecasted results
    result_text.insert(tk.END, forecast_df.to_string(index=False))
    
    # Reset all variables to their initial states after the forecast
    reset_variables()

# GUI setup
root = tk.Tk()
root.title("Gold Price Forecast System")

# Label for selecting a date
date_label = tk.Label(root, text="Select Date for Forecast:")
date_label.pack(pady=10)

# Calendar widget for date selection
cal = Calendar(root, selectmode='day', date_pattern='yyyy-mm-dd')
cal.pack(pady=10)

# Submit button
submit_btn = ttk.Button(root, text="Submit", command=submit_forecast)
submit_btn.pack(pady=10)

# Text box to display the forecast results
result_text = tk.Text(root, height=70, width=100)
result_text.pack(pady=10)

# Initialize the models and variables for the first time
reset_variables()

# Start the Tkinter event loop
root.mainloop()


2024-10-03 15:28:45,426 - INFO - Prophet model loaded successfully.
2024-10-03 15:28:45,694 - INFO - XGBoost model loaded successfully.
2024-10-03 15:28:45,702 - INFO - Scaler loaded successfully.
2024-10-03 15:28:45,702 - INFO - Regressors loaded successfully: ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_futures', 'effr']
2024-10-03 15:38:57,315 - WARNING - Feature mismatch: Expected 98, but got 97.
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardSca